In [1]:
%pip install peft

  Obtaining dependency information for peft from https://files.pythonhosted.org/packages/8b/1b/aee2a330d050c493642d59ba6af51f3910cb138ea48ede228c84c204a5af/peft-0.7.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 1.6 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install evaluate

  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 920.0 kB/s eta 0:00:000:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install jiwer

  Obtaining dependency information for jiwer from https://files.pythonhosted.org/packages/0d/4f/ee537ab20144811dd99321735ff92ef2b3a3230b77ed7454bed4c44d21fc/jiwer-3.0.3-py3-none-any.whl.metadata


In [6]:
from datasets import load_dataset
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, GenerationConfig, TrainingArguments, Trainer
from PIL import Image
import torch
import requests
import accelerate
from peft import LoftQConfig, LoraConfig, get_peft_model
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchvision.transforms import v2
from torchvision.io import read_image
import matplotlib.pyplot as plt 
import plotly.express as px
import os
from transformers import AdamW
from tqdm.notebook import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning

In [7]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [8]:
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-small-printed')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-small-printed').to("cuda:0")

preprocessor_config.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.21k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/246M [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
img = read_image('/kaggle/input/valid/images/0189O07_1__png_jpg.rf.4176dae33b50eae973fc181f33206aa9.jpg')
fig = px.imshow(img.permute(1,2,0))
fig.show()

In [10]:
pixel_values = processor(img, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values.to(device), max_new_tokens = 10)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
generated_text

'TOTAL SALE'

In [11]:
from evaluate import load
cer_metric = load("cer")

def compute_cer(pred_ids, label_ids):
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return cer

In [12]:
def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [13]:
train = os.listdir('/kaggle/input/train/images')
valid = os.listdir('/kaggle/input/valid/images')
test = os.listdir('/kaggle/input/test/images')

In [14]:
class CustomDataset(Dataset):
    def __init__(self, filenames, processor, prefix=''):
        self.filenames = filenames
        self.processor = processor
        self.prefix = prefix

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        image_path = '/kaggle/input/' + self.prefix + '/images/' + self.filenames[idx]
        label = self.filenames[idx].split("_")[0]
        img_arr = read_image(image_path)

        pixel_values = self.processor(img_arr, return_tensors="pt").pixel_values

        labels = self.processor.tokenizer(
            label,
            padding="max_length",
            max_length=7 #max_length for a NA license plate
        ).input_ids
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

def collate_fn(batch):
    pixel_values_batch = [item["pixel_values"] for item in batch]
    labels_batch = [item["labels"] for item in batch]

    # Pad sequences to the length of the longest sequence in the batch
    pixel_values_batch = torch.nn.utils.rnn.pad_sequence(pixel_values_batch, batch_first=True)
    labels_batch = torch.nn.utils.rnn.pad_sequence(labels_batch, batch_first=True, padding_value=-100)

    return {"pixel_values": pixel_values_batch, "labels": labels_batch}

# Example usage:
# train_dataset = CustomDataset(filenames=train_filenames, processor=my_processor)
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn)


In [15]:
train_ds = CustomDataset(train, prefix = 'train', processor = processor)
valid_ds = CustomDataset(valid, prefix = 'valid', processor = processor)
test_ds = CustomDataset(test, prefix = 'test', processor = processor)

In [16]:
batch_size = 4

In [17]:
train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_dataloader = DataLoader(valid_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [18]:
print("Number of training examples:", len(train_ds))
print("Number of validation examples:", len(valid_ds))
print("Number of test examples:", len(test_ds))

Number of training examples: 1843
Number of validation examples: 527
Number of test examples: 263


In [19]:
encoding = train_ds[0]
for k,v in encoding.items():
    print(k, v.shape)

pixel_values torch.Size([3, 384, 384])
labels torch.Size([7])


In [20]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 10 # NA License plate is 6-7 digits Max
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [21]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

## Without Finetuning

In [42]:
print_trainable_parameters(model)

trainable params: 61596672 || all params: 61596672 || trainable%: 100.00


In [37]:
# evaluate
model.eval()
valid_cer = 0.0
with torch.no_grad():
    for batch in tqdm(test_dataloader):
        # run batch generation
        outputs = model.generate(batch["pixel_values"].to(device))
        # compute metrics
        cer = compute_cer(pred_ids=outputs, label_ids=batch["labels"])
        valid_cer += cer 

    print("Validation CER:", valid_cer / len(test_dataloader))

  0%|          | 0/66 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1518: UserWarning:

You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )



Validation CER: 1.976731601731602


Performance is not so good.

In [38]:
model = model.to("cuda:0")

In [39]:
optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(10):  # loop over the dataset multiple times
   # train
    model.train()
    train_loss = 0.0
    for batch in tqdm(train_dataloader):
        # get the inputs
        for k,v in batch.items():
            batch[k] = v.to("cuda:0")

        # forward + backward + optimize
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_loss += loss.item()

    print(f"Loss after epoch {epoch}:", train_loss/len(train_dataloader))
    
   # evaluate
    model.eval()
    valid_cer = 0.0
    with torch.no_grad():
        for batch in tqdm(valid_dataloader):
            # run batch generation
            outputs = model.generate(batch["pixel_values"].to("cuda:0"))
            # compute metrics
            cer = compute_cer(pred_ids=outputs, label_ids=batch["labels"])
            valid_cer += cer 

        print("Validation CER:", valid_cer / len(valid_dataloader))

model.save_pretrained(".")

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning:

This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning



  0%|          | 0/461 [00:00<?, ?it/s]

Loss after epoch 0: 1.5026635132616875


  0%|          | 0/132 [00:00<?, ?it/s]

Validation CER: 0.1508838383838384


...

Loss after epoch 4: 0.38179712563171303


  0%|          | 0/132 [00:00<?, ?it/s]

Validation CER: 0.11453823953823955


  0%|          | 0/461 [00:00<?, ?it/s]

KeyboardInterrupt: 

# LoRA

In [22]:
config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    target_modules=["query","value"],
)

In [23]:
model = get_peft_model(model, config)

In [24]:
model = model.to("cuda:1")

In [25]:
print_trainable_parameters(model)

trainable params: 589824 || all params: 62186496 || trainable%: 0.95


In [26]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
epochs = 100

In [27]:
batch_size = 16

In [28]:
train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_dataloader = DataLoader(valid_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [29]:
for epoch in range(epochs):  # loop over the dataset multiple times
   # train
    model.train()
    train_loss = 0.0
    for batch in tqdm(train_dataloader):
      # get the inputs
        for k,v in batch.items():
            batch[k] = v.to("cuda:1")

      # forward + backward + optimize
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_loss += loss.item()

    print(f"Loss after epoch {epoch}:", train_loss/len(train_dataloader))
    
   # evaluate
    model.eval()
    valid_cer = 0.0
    with torch.no_grad():
        for batch in tqdm(valid_dataloader):
       # run batch generation
            outputs = model.generate(batch["pixel_values"].to("cuda:1"))
       # compute metrics
            cer = compute_cer(pred_ids=outputs, label_ids=batch["labels"])
            valid_cer += cer 

        print("Validation CER:", valid_cer / len(valid_dataloader))

model.save_pretrained(".")

  0%|          | 0/116 [00:00<?, ?it/s]

Loss after epoch 0: 8.062421091671649


  0%|          | 0/33 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1518: UserWarning:

You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )



Validation CER: 0.8026334776334778


...

Loss after epoch 48: 0.2105239414577854


  0%|          | 0/33 [00:00<?, ?it/s]

Validation CER: 0.03003246753246753


  0%|          | 0/116 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [30]:
model.eval()
test_cer = 0.0
with torch.no_grad():
    for batch in tqdm(test_dataloader):
   # run batch generation
        outputs = model.generate(batch["pixel_values"].to("cuda:1"))
   # compute metrics
        cer = compute_cer(pred_ids=outputs, label_ids=batch["labels"])
        test_cer += cer 

  0%|          | 0/17 [00:00<?, ?it/s]

In [31]:
print("Test CER:", test_cer / len(test_dataloader))

Test CER: 0.028286314525810326


In [32]:
model.save_pretrained("/kaggle/working/adapter")

In [ ]:
torch.save(model.state_dict(), '/kaggle/working/model.pth')